<a href="https://colab.research.google.com/github/Chpppy/Colab/blob/main/notebooks/PixelCopter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-5ma4jz2t
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-5ma4jz2t
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/simoninithomas/gym-games to /tmp/pip-req-build-6v8c858p
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /tmp/pip-req-build-6v8c858p
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 10.1 MB/s eta 0:00:00
  Created wheel for ple: filename=ple-0.0.1-py3-none-any.whl size=50772 sha256=e23aca282657939d4743531b7741b66fa70a7fc2b1e5eb7ba263043e4410262e
  Stored in dir

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import gym
import gym_pygame

import numpy as np
from collections import deque

In [12]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [13]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", state_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", action_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

The State Space is:  7
Sample observation [ 0.70900345 -1.244844   -0.23239377 -0.15580711  0.19904383 -0.8210153
 -0.33501986]

 _____ACTION SPACE_____ 

The Action Space is:  2
Action Space Sample 0


In [14]:
class Policy(nn.Module):
    def __init__(self, state_size, hidden_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        tensor_state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(tensor_state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [25]:
param = {
    "state_size" : state_size,
    "hidden_size" : 32,
    "action_size" : action_size,
    "total_training_episode" : 10_000,
    "max_step" : 1000,
    "lr" : 1e-3,
    "gamma" : 0.99
}

In [22]:
PixelPolicy = Policy(param["state_size"], param["hidden_size"], param["action_size"]).to(device)
optimizer = torch.optim.Adam(PixelPolicy.parameters(), param["lr"])

In [23]:
def reinforce(env, optimizer, max_step, total_training_episode, gamma, print_time):
    scores = deque(maxlen=100)
    score = []
    for episode in range(1, total_training_episode + 1):
        rewards = []
        log_probs = []
        state = env.reset()
        for i in range(max_step):
            action, log_prob = PixelPolicy.act(state)
            log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break

        scores.append(sum(rewards))
        score.append(sum(rewards))
        cumulative_reward = deque(maxlen=max_step + 1)
        cumulative_reward.append(0)

        for i in range(len(rewards))[::-1]:
            cumulative_reward.appendleft(rewards[i] + gamma * cumulative_reward[0])

        cumulative_reward = torch.tensor(cumulative_reward)
        eps = np.finfo(np.float32).eps.item()

        cumulative_reward = (cumulative_reward - cumulative_reward.mean()) / (cumulative_reward.std() + eps)

        policy_loss = []
        for log_prob, disc_return in zip(log_probs, cumulative_reward):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if episode % print_time == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores)))

    return score

In [26]:
score = reinforce(env, optimizer, param["max_step"], param["total_training_episode"], param["gamma"], 100)

Episode 100	Average Score: 7.82
Episode 200	Average Score: 12.38
Episode 300	Average Score: 10.72
Episode 400	Average Score: 10.79
Episode 500	Average Score: 10.08
Episode 600	Average Score: 12.84
Episode 700	Average Score: 13.62
Episode 800	Average Score: 13.94
Episode 900	Average Score: 13.03
Episode 1000	Average Score: 13.05
Episode 1100	Average Score: 13.83
Episode 1200	Average Score: 15.09
Episode 1300	Average Score: 11.70
Episode 1400	Average Score: 14.68
Episode 1500	Average Score: 15.78
Episode 1600	Average Score: 16.37
Episode 1700	Average Score: 18.49
Episode 1800	Average Score: 13.91
Episode 1900	Average Score: 16.26
Episode 2000	Average Score: 15.28
Episode 2100	Average Score: 14.76
Episode 2200	Average Score: 16.55
Episode 2300	Average Score: 14.44
Episode 2400	Average Score: 14.43
Episode 2500	Average Score: 15.85
Episode 2600	Average Score: 12.88
Episode 2700	Average Score: 19.75
Episode 2800	Average Score: 18.13
Episode 2900	Average Score: 17.65
Episode 3000	Average Sco

KeyboardInterrupt: 

In [28]:
torch.save(PixelPolicy.state_dict(), "Pixelcopter-PLE.pth")